In [ ]:
from glob import glob
path='../'
filename='AFEP2-02.fepout'
fepoutFiles = glob(path+filename)
temperature = 300
RT = 0.00198720650096 * temperature
print(f"Will process {len(fepoutFiles)} fepout files.")

# Import necessary libraries
SAFEP_parse.py contains all the functions and library calls necessary to run the notebook
# Required libraries:
- numpy
- pandas
- matplotlib
- alchemlyb (pip install git+https://github.com/alchemistry/alchemlyb)
- natsort (for sorting file names)
- glob (for unix-like file paths)



In [ ]:
from SAFEP_parse import *
plt.rcParams['figure.dpi'] = 150

# Use the Multistate Bennett Acceptance Ratio estimator

In [ ]:
u_nk = namd.extract_u_nk(fepoutFiles, temperature)
bar = BAR()
bar.fit(u_nk)

# Extract key features from the MBAR fitting

In [ ]:
l, l_mid, f, df, ddf, errors = get_MBAR(bar)
#print(df.cumsum() * RT) #in unit of KT
print("Overall free energy",df.cumsum() * RT) #in unit of kcal/mol

print("Errors", errors)

In [ ]:
#Overall delta G_site
print(f'\u0394G*(site) = {np.round((df.cumsum()*RT)[-1], 3)}\u00B1{np.round(errors[-1], 3)} kcal/mol')

# Plot the change in free energy based on MBAR estimates

In [ ]:
# Cumulative change in kT
plt.errorbar(l, f, yerr=errors, marker='.')
plt.xlabel('lambda')
plt.ylabel('DeltaG(lambda) (kT)')
plt.show()

# Cumulative change in kcal/mol
plt.errorbar(l, f * RT, yerr=errors, marker='.')
plt.xlabel('lambda')
plt.ylabel('DeltaG(lambda)(kcal/mol)')
plt.show()

# Per-window change in kT
plt.errorbar(l_mid, df, yerr=ddf, marker='.')
plt.xlabel('lambda')
plt.ylabel('Delta G per window (kT)')
plt.show()


# Plot the estimated total change in free energy as a function of simulation time; contiguous subsets starting at t=0 ("Forward") and t=end ("Reverse")

In [ ]:
convergence_plot(u_nk, l)

# Use an exponential estimator to assess residual discrepancies and check for hysteresis

In [ ]:
l, l_mid, dG_f, dG_b = get_EXP(u_nk)

In [ ]:
plt.vlines(l_mid, np.zeros(len(l_mid)), dG_f + np.array(dG_b), label="fwd - bwd", linewidth=3)

plt.legend()
plt.title('Fwd-bwd discrepancies by lambda')
plt.xlabel('Lambda')
plt.ylabel('Diff. in delta-G')

# The above data should follow a roughly normal distribution centered on 0.

In [ ]:
plt.hist(dG_f + np.array(dG_b));
plt.title('Distribution of fwd-bwd discrepancies')
plt.xlabel('Difference in delta-G')
plt.ylabel('Count')